In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import os, json
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

In [12]:
# ==============================================================================
# Configuration Constants
# ==============================================================================
# --- Paths ---
# NOTE: Adjust these paths to your environment
DATASET_PATH = "../TestDataSet"  # Path to the original dataset (ImageFolder structure)
LABEL_JSON_PATH = os.path.join(DATASET_PATH, "labels_list.json") # Path to the JSON file mapping folder order to ImageNet indices
ADV_DATASET_PATH_TASK1 = "../Adversarial_Test_Set_1" # Output directory for PGD adversarial images
ADV_DATASET_PATH_TASK2 = "../AdversarialTestSet2_PGD_Refactored" # Output directory for PGD adversarial images
ADV_DATASET_PATH_TASK3 = "../Adversarial_Test_Set_3" # Output directory for PGD adversarial images

# --- Model & Device ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WEIGHTS = 'IMAGENET1K_V1'
# --- PGD Attack Parameters ---
# Epsilon: Max L-infinity perturbation allowed (relative to [0, 1] pixel range)
EPSILON = 0.02
# Alpha: Step size for each PGD iteration (often eps / iters or smaller)
PGD_ALPHA = 0.005 # As provided in user's code
# Iterations: Number of PGD steps
PGD_ITERS = 5     # As provided in user's code

# --- Dataloader Parameters ---
BATCH_SIZE = 32  # Adjusted for potentially faster generation/evaluation
NUM_WORKERS = 0  # Adjust based on your system


In [13]:

with open(LABEL_JSON_PATH, "r") as f:
    label_lines = json.load(f)
true_imagenet_indices = [int(line.split(":")[0]) for line in label_lines]

In [14]:
orig_folder = torchvision.datasets.ImageFolder(root=DATASET_PATH, transform=None)
folder_to_imagenet_index = {
    class_name: true_imagenet_indices[i]
    for i, class_name in enumerate(orig_folder.classes)
}

In [15]:
orig_folder.samples = [
    (path, folder_to_imagenet_index[os.path.basename(os.path.dirname(path))])
    for path, _ in orig_folder.samples
]

In [16]:
orig_folder.samples.sort(key=lambda x: x[0])
orig_labels = [lbl for _, lbl in orig_folder.samples]

In [17]:
class AdvDataset(Dataset):
    def __init__(self, image_dir, true_labels, transform=None):
        self.image_paths = sorted(
            os.path.join(image_dir, fn)
            for fn in os.listdir(image_dir) if fn.endswith(".png")
        )
        assert len(self.image_paths) == len(true_labels), (
            f"Found {len(self.image_paths)} images but {len(true_labels)} labels"
        )
        self.labels = true_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = torchvision.models.densenet121(weights='IMAGENET1K_V1')
new_model.eval().to(device)

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

paths = {
    "Original": DATASET_PATH,
    "FGSM":     ADV_DATASET_PATH_TASK1,
    "PGD":      ADV_DATASET_PATH_TASK2,
    "Patch":    ADV_DATASET_PATH_TASK3,
}


In [19]:
def evaluate(model, loader):
    top1 = top5 = total = 0
    with torch.no_grad():
        for imgs, lbls in tqdm(loader):
            imgs, lbls = imgs.to(device), lbls.to(device)
            out = model(imgs)
            _, t1 = out.topk(1, dim=1)
            _, t5 = out.topk(5, dim=1)
            top1 += (t1.squeeze() == lbls).sum().item()
            top5 += sum([lbls[i] in t5[i] for i in range(len(lbls))])
            total += imgs.size(0)
    return 100*top1/total, 100*top5/total

In [20]:
results = {}
for name, p in paths.items():
    if name == "Original":
        ds = orig_folder  # already has transform=None; now attach transform
        ds.transform = transform
    else:
        ds = AdvDataset(p, orig_labels, transform)
    loader = DataLoader(ds, batch_size=32, shuffle=False)
    print(f"\n📊 Evaluating {name} dataset on DenseNet-121:")
    t1, t5 = evaluate(new_model, loader)
    results[name] = (t1, t5)
    print(f"→ Top-1: {t1:.2f}%   Top-5: {t5:.2f}%")


📊 Evaluating Original dataset on DenseNet-121:


100%|██████████| 16/16 [00:02<00:00,  7.77it/s]


→ Top-1: 74.60%   Top-5: 93.60%

📊 Evaluating FGSM dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.39it/s]


→ Top-1: 43.00%   Top-5: 66.40%

📊 Evaluating PGD dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.37it/s]


→ Top-1: 39.40%   Top-5: 64.20%

📊 Evaluating Patch dataset on DenseNet-121:


100%|██████████| 16/16 [00:06<00:00,  2.43it/s]

→ Top-1: 41.80%   Top-5: 65.80%
